In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers accelerate torch torchvision bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 15.9 MB/s eta 0:00:00


In [2]:
from transformers import Blip2Processor, Blip2ForConditionalGeneration
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

processor = Blip2Processor.from_pretrained(
    "Salesforce/blip2-flan-t5-xl"
)

model = Blip2ForConditionalGeneration.from_pretrained(
    "Salesforce/blip2-flan-t5-xl",
    torch_dtype=torch.float16
).to(device)

model.eval()


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The image processor of type `BlipImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. 


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/1289 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie language_model.shared.weight to language_model.lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


Blip2ForConditionalGeneration(
  (vision_model): Blip2VisionModel(
    (embeddings): Blip2VisionEmbeddings(
      (patch_embedding): Conv2d(3, 1408, kernel_size=(14, 14), stride=(14, 14))
    )
    (encoder): Blip2Encoder(
      (layers): ModuleList(
        (0-38): 39 x Blip2EncoderLayer(
          (self_attn): Blip2Attention(
            (qkv): Linear(in_features=1408, out_features=4224, bias=True)
            (projection): Linear(in_features=1408, out_features=1408, bias=True)
          )
          (layer_norm1): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
          (mlp): Blip2MLP(
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=1408, out_features=6144, bias=True)
            (fc2): Linear(in_features=6144, out_features=1408, bias=True)
          )
          (layer_norm2): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
        )
      )
    )
    (post_layernorm): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
  )
  (qf

In [5]:
def generate_raw_caption(image):
    inputs = processor(
        images=image,
        return_tensors="pt"
    ).to(device)

    output = model.generate(
        **inputs,
        max_new_tokens=60,
        do_sample=False
    )

    return processor.decode(output[0], skip_special_tokens=True)


In [6]:
image_files = [
    "photo_0001.jpg",
    "photo_0002.jpg",
    "photo_0003.jpg",
    "photo_0004.jpg",
    "photo_0005.jpg",
    "photo_0006.jpg",
    "photo_0007.jpg"
]
print(image_files)

['photo_0001.jpg', 'photo_0002.jpg', 'photo_0003.jpg', 'photo_0004.jpg', 'photo_0005.jpg', 'photo_0006.jpg', 'photo_0007.jpg']


In [7]:
from google import genai
from google.genai import types

# 1. Setup Client
client = genai.Client(api_key="YOUR_GEMINI_API_KEY_HERE")

def rewrite_caption_french_cloud(raw_caption):
    # Your specific RAG rules
    rules = """
RÈGLES DE GÉNÉRATION DE LÉGENDES

1. Format
- La sortie doit contenir une seule phrase.
- La phrase doit être rédigée au présent de l’indicatif.
- La longueur maximale est de 50 mots.

2. Contenu autorisé
- Décrire uniquement les éléments directement visibles dans l’image.
- Ne pas formuler d’hypothèses, d’interprétations ou de déductions.
- Éviter toute expression d’incertitude (ex. : probablement, semble, pourrait).

3. Structure obligatoire
- La description doit suivre l’ordre suivant :
  sujet principal → action observable → contexte (lieu, objets visibles).
- Cet ordre ne doit pas être modifié.

4. Action
- Décrire uniquement des actions clairement observables.
- En l’absence d’action évidente, utiliser une description statique
  (ex. : est debout, est assis, se tient immobile).

5. Ton et style
- Employer un ton neutre, factuel et objectif.
- Utiliser un vocabulaire simple et descriptif.
- Éviter les adjectifs subjectifs ou évaluatifs.

6. Restrictions
- Ne pas inclure de descriptions graphiques ou choquantes
  (blessures détaillées, sang, violence explicite).
- Ne pas mentionner d’éléments hors champ ou non visibles.
- Ne pas identifier des personnes, marques ou entités spécifiques.


    """

    prompt = f"{rules}\n\nTraduit cette phrase : {raw_caption}"

    # Gemini 2.5 Flash is incredibly fast and free
    response = client.models.generate_content(
        model="gemini-2.5-flash",
        contents=prompt
    )

    return response.text.strip()

# Test run
print(rewrite_caption_french_cloud("a man is cooking food on a grill"))
# Expected: "Un homme fait cuire des aliments sur un gril."

Un homme cuisine de la nourriture sur un gril.


In [8]:
from PIL import Image

for filename in image_files:
    image_path = f"/content/{filename}"
    image = Image.open(image_path).convert("RGB")
    raw_caption = generate_raw_caption(image)
    final_caption = rewrite_caption_french_cloud(raw_caption)

    print(f"Image: {filename}")
    print(f"Raw Caption: {raw_caption}")
    print(f"Caption: {final_caption}\n")


ClientError: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-2.5-flash\nPlease retry in 3.282404438s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerDayPerProjectPerModel-FreeTier', 'quotaDimensions': {'model': 'gemini-2.5-flash', 'location': 'global'}, 

In [ ]:
!mkdir -p /content/coco
!wget -q http://images.cocodataset.org/zips/val2014.zip
!unzip -q val2014.zip -d /content/coco


In [15]:
!wget -q http://images.cocodataset.org/annotations/annotations_trainval2014.zip
!unzip -q annotations_trainval2014.zip -d /content/coco


In [16]:
import json
from PIL import Image
import os

ANN_PATH = "/content/coco/annotations/captions_val2014.json"
IMG_DIR = "/content/coco/val2014"

with open(ANN_PATH, "r") as f:
    coco_data = json.load(f)

# Build image_id → captions mapping
imgid_to_captions = {}
for ann in coco_data["annotations"]:
    imgid_to_captions.setdefault(ann["image_id"], []).append(ann["caption"])

# Image metadata
imgid_to_filename = {
    img["id"]: img["file_name"]
    for img in coco_data["images"]
}

print("Images:", len(imgid_to_filename))
print("Captions per image:", len(next(iter(imgid_to_captions.values()))))


Images: 40504
Captions per image: 5


In [17]:
from tqdm import tqdm
import torch

MAX_WORDS = 50
results = []

def truncate(text, max_words=50):
    return " ".join(text.split()[:max_words])

image_ids = list(imgid_to_filename.keys())[:1000]  # start with 1k

for img_id in tqdm(image_ids):
    img_path = os.path.join(IMG_DIR, imgid_to_filename[img_id])
    image = Image.open(img_path).convert("RGB")

    inputs = processor(images=image, return_tensors="pt").to(device)

    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=MAX_WORDS
        )

    caption = processor.decode(
        output_ids[0],
        skip_special_tokens=True
    )

    results.append({
        "image_id": img_id,
        "generated": truncate(caption),
        "references": imgid_to_captions[img_id]
    })


100%|██████████| 1000/1000 [11:34<00:00,  1.44it/s]


In [18]:
import json

SAVE_PATH = "/content/drive/MyDrive/AICV_captioning/blip2_baseline_captions.json"

with open(SAVE_PATH, "w") as f:
    json.dump(results, f, indent=2)

print("Saved:", SAVE_PATH)


Saved: /content/drive/MyDrive/AICV_captioning/blip2_baseline_captions.json


In [21]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.6 MB/s eta 0:00:00


In [23]:
!git clone https://github.com/cocodataset/cocoapi.git
!pip install -q pycocotools


Cloning into 'cocoapi'...
remote: Enumerating objects: 975, done.
remote: Total 975 (delta 0), reused 0 (delta 0), pack-reused 975 (from 1)
Receiving objects: 100% (975/975), 11.72 MiB | 15.61 MiB/s, done.
Resolving deltas: 100% (576/576), done.


In [25]:
!pip install pycocoevalcap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.3/104.3 MB 1.7 MB/s eta 0:00:00


In [26]:
PRED_PATH = "/content/blip2_predictions.json"

predictions = [
    {
        "image_id": r["image_id"],
        "caption": r["generated"]
    }
    for r in results
]

with open(PRED_PATH, "w") as f:
    json.dump(predictions, f)

print("Saved predictions:", PRED_PATH)


Saved predictions: /content/blip2_predictions.json


In [27]:
import evaluate

bleu = evaluate.load("bleu")
meteor = evaluate.load("meteor")

preds = [r["generated"] for r in results]
refs = [r["references"] for r in results]

bleu1 = bleu.compute(predictions=preds, references=refs, max_order=1)["bleu"]
bleu4 = bleu.compute(predictions=preds, references=refs, max_order=4)["bleu"]
meteor_score = meteor.compute(predictions=preds, references=refs)["meteor"]


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [4]:
import json

RESULTS_PATH = "/content/drive/MyDrive/AICV_captioning/blip2_baseline_captions.json"

with open(RESULTS_PATH, "r") as f:
    results = json.load(f)

print("Loaded results:", len(results))
print(results[0].keys())


Loaded results: 1000
dict_keys(['image_id', 'generated', 'references'])


In [6]:
preds = [r["generated"] for r in results]
refs = [r["references"] for r in results]

print(len(preds), len(refs))


1000 1000


In [7]:
PRED_PATH = "/content/blip2_predictions.json"

predictions = [
    {
        "image_id": r["image_id"],
        "caption": r["generated"]
    }
    for r in results
]

with open(PRED_PATH, "w") as f:
    json.dump(predictions, f)

print("Saved:", PRED_PATH)


Saved: /content/blip2_predictions.json


In [8]:
ANN_PATH = "/content/coco/annotations/captions_val2014.json"

with open(ANN_PATH, "r") as f:
    coco_gt = json.load(f)

eval_image_ids = set(r["image_id"] for r in results)

filtered_images = [
    img for img in coco_gt["images"]
    if img["id"] in eval_image_ids
]

filtered_annotations = [
    ann for ann in coco_gt["annotations"]
    if ann["image_id"] in eval_image_ids
]

filtered_coco_gt = {
    "info": coco_gt.get("info", {}),
    "licenses": coco_gt.get("licenses", []),
    "images": filtered_images,
    "annotations": filtered_annotations
}

FILTERED_GT_PATH = "/content/captions_val2014_filtered.json"

with open(FILTERED_GT_PATH, "w") as f:
    json.dump(filtered_coco_gt, f)

print("Filtered images:", len(filtered_images))
print("Filtered annotations:", len(filtered_annotations))


Filtered images: 1000
Filtered annotations: 5005


In [9]:
!pip install -q evaluate
import evaluate

bleu = evaluate.load("bleu")
meteor = evaluate.load("meteor")

bleu1 = bleu.compute(predictions=preds, references=refs, max_order=1)["bleu"]
bleu4 = bleu.compute(predictions=preds, references=refs, max_order=4)["bleu"]
meteor_score = meteor.compute(predictions=preds, references=refs)["meteor"]


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [10]:
!pip install -q pycocotools pycocoevalcap

from pycocotools.coco import COCO
from pycocoevalcap.eval import COCOEvalCap

coco = COCO(FILTERED_GT_PATH)
coco_res = coco.loadRes(PRED_PATH)

coco_eval = COCOEvalCap(coco, coco_res)
coco_eval.evaluate()

cider_score = coco_eval.eval["CIDEr"]


loading annotations into memory...
Done (t=0.03s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.01s)
creating index...
index created!
tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 7825, 'reflen': 8885, 'guess': [7825, 6825, 5825, 4825], 'correct': [6112, 3252, 1440, 562]}
ratio: 0.8806978052897151
Bleu_1: 0.682
Bleu_2: 0.533
Bleu_3: 0.394
Bleu_4: 0.281
computing METEOR score...
METEOR: 0.244
computing Rouge score...
ROUGE_L: 0.539
computing CIDEr score...
CIDEr: 1.008
computing SPICE score...


CalledProcessError: Command '['java', '-jar', '-Xmx8G', 'spice-1.0.jar', '/usr/local/lib/python3.12/dist-packages/pycocoevalcap/spice/tmp/tmpcgh2wb4k', '-cache', '/usr/local/lib/python3.12/dist-packages/pycocoevalcap/spice/cache', '-out', '/usr/local/lib/python3.12/dist-packages/pycocoevalcap/spice/tmp/tmpwsinx_ju', '-subset', '-silent']' returned non-zero exit status 1.

In [11]:
metrics = {
    "BLEU-1": bleu1,
    "BLEU-4": bleu4,
    "METEOR": meteor_score,
    "CIDEr": cider_score
}

metrics


NameError: name 'cider_score' is not defined